Firstly get all variables we are considering appended to a raw dataframe. 
Work on the 2visit first as its the largest.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ast


non_prog = pd.read_csv('Cohorts\\NonProgressors\\cleaned_20visit_nonprogressor.csv')
prog = pd.read_csv('Cohorts\\Progressors\\cleaned_20visit_progressors.csv')

# Read the CSV files and extract IDs
non_prog_ids = non_prog['NACCID'].tolist()
prog_ids = prog['NACCID'].tolist()
progvecs = prog.iloc[:, 1:].values
nonprogvecs = non_prog.iloc[:, 1:].values

# Create a new dataframe with IDs and labels
ids = non_prog_ids + prog_ids
labels = [0] * len(non_prog_ids) + [1] * len(prog_ids)
vectors = np.concatenate((nonprogvecs, progvecs), axis=0)

data = []
for id, label, vector in zip(ids, labels, vectors):
    for visit, value in enumerate(vector, start=1):
        data.append((id, visit, label, value))

df_combined = pd.DataFrame(data, columns=['ID', 'Visit', 'Prog_ID', 'Progression'])

# Set MultiIndex
#df_combined.set_index(['ID', 'Visit'], inplace=True)

# Print the DataFrame to verify
print(df_combined)

df_combined.to_csv('Cohorts\\20visit_combined.csv')

            ID  Visit  Prog_ID  \
0   NACC263960      1        0   
1   NACC276815      1        0   
2   NACC303378      1        0   
3   NACC642177      1        0   
4   NACC038012      1        0   
5   NACC147305      1        0   
6   NACC270893      1        0   
7   NACC378691      1        0   
8   NACC621217      1        0   
9   NACC687774      1        0   
10  NACC961626      1        0   
11  NACC227022      1        0   
12  NACC261581      1        0   
13  NACC668158      1        0   

                                          Progression  
0   (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1   (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2   (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3   (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
4   (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
5   (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
6   (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
7   (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

1-27-25
Right now there is an issue where the transition from categories (like 0,0 to 0,1) are being malformed. I am going to just manually remove for now as a quick fix because it is a little overcomplicated to handle up to 20 different vector combinations with basic syntax checking for now. 


In [24]:
def add_columns_from_uds(df_combined, uds_df):
    # Initialize the new columns with default values
    df_combined['SEX'] = None
    df_combined['EDUC'] = None
    
    # Iterate through the IDs in the first column of df_combined
    for index, row in df_combined.iterrows():
        id = row['ID']
        sex = uds_df.loc[uds_df['NACCID'] == id, 'SEX'].values
        educ = uds_df.loc[uds_df['NACCID'] == id, 'EDUC'].values
        
        # Update the new columns with the corresponding values from uds_df
        df_combined.at[index, 'SEX'] = sex[0] if len(sex) > 0 else None
        df_combined.at[index, 'EDUC'] = educ[0] if len(educ) > 0 else None
    
    return df_combined

# Load the global uds.csv DataFrame
uds_df = pd.read_csv('uds.csv', low_memory=False)
# Add new columns to the combined DataFrame
df_combined = add_columns_from_uds(df_combined, uds_df)

# Print the updated DataFrame to verify
print(df_combined)

              ID  Visit  Prog_ID Progression SEX EDUC
0     NACC141167      1        0   (0, 0, 0)   2   12
1     NACC310984      1        0   (0, 0, 0)   1   14
2     NACC591136      1        0   (0, 0, 0)   2   18
3     NACC665925      1        0   (0, 0, 0)   2   16
4     NACC701804      1        0   (0, 0, 0)   1   16
...          ...    ...      ...         ...  ..  ...
2268  NACC330027      1        1   (1, 2, 1)   2   13
2269  NACC879413      1        1   (1, 2, 1)   2   12
2270  NACC530751      1        1   (1, 2, 1)   2   18
2271  NACC853772      1        1   (1, 2, 1)   1   12
2272  NACC977631      1        1   (1, 2, 1)   1   16

[2273 rows x 6 columns]
